In [1]:
from simulator import *
import numpy as np
import functools

In [2]:
import numpy as np
import IPython
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit
from qiskit.visualization import plot_histogram
from qiskit_aer import AerSimulator

import seaborn as sns
sns.set()

In [3]:
def x_minus_f_x(n, f:list[bool]):
    pos = []
    for i, v in enumerate(f):
        if v:
            pos.append(i)
    print(pos)
    return cnnot(n+1,pos)

def test_cnnot(n, f:list[bool]):
    cnn = x_minus_f_x(n,f)
    for x in range(2**n):
        inp_mat = [(x & 1<<i) >>i for i in range(n)] 
        print(cnn @ inp_mat)

def cnnot(n, pos: list[int]):
    assert all([i < n-1 for i in pos])
    KETBRA_1 = KET_1 @ BRA_1
    KETBRA_0 = KET_0 @ BRA_0

    do_nothing = []
    for i in pos:
        state = [I for _ in range(n)]
        state[i] = KETBRA_0
        do_nothing += [functools.reduce(lambda x,y: np.kron(x,y), state)]
    
    flip_state = [I for _ in range(n-1)] + [X]
    for i in pos:
        flip_state[i] = KETBRA_1
    flip = [functools.reduce(lambda x,y: np.kron(x,y), flip_state)]

    val = np.array(functools.reduce(lambda x,y: x+y, do_nothing+flip), dtype=int)

    for i in range(val.shape[0]):
        for j in range(val.shape[1]):
            val[i,j] = 1 if val[i,j] > 0 else 0

    return val

In [22]:
test = cnnot(5,[1,2])
test.shape
after_h = functools.reduce(lambda x,y: np.kron(x,y), map(lambda x: H @ x, [KET_0 for _ in range(4)]+[X @ KET_0]))
print(after_h)
after_oracle = test @ after_h
H_n = functools.reduce(lambda x,y: np.kron(x,y), [H for _ in range(4+1)])
pre_measure = H_n @ after_oracle
print(pre_measure)
print(pre_measure.shape)
MINUS = H@ (X@KET_0)
print(MINUS)
pre_measure.reshape((16,2))
pre_measure = pre_measure/MINUS.T
print(pre_measure)
for i in range(4):
    print(pre_measure[1<<i])
a = np.zeros(4,dtype=complex)
for idx,v in enumerate(pre_measure):
    for i in range(4):
        if (idx & 1<<i):
            a[i]+=v*v


[[ 0.1767767+0.j]
 [-0.1767767+0.j]
 [ 0.1767767+0.j]
 [-0.1767767+0.j]
 [ 0.1767767+0.j]
 [-0.1767767+0.j]
 [ 0.1767767+0.j]
 [-0.1767767+0.j]
 [ 0.1767767+0.j]
 [-0.1767767+0.j]
 [ 0.1767767+0.j]
 [-0.1767767+0.j]
 [ 0.1767767+0.j]
 [-0.1767767+0.j]
 [ 0.1767767+0.j]
 [-0.1767767+0.j]
 [ 0.1767767+0.j]
 [-0.1767767+0.j]
 [ 0.1767767+0.j]
 [-0.1767767+0.j]
 [ 0.1767767+0.j]
 [-0.1767767+0.j]
 [ 0.1767767+0.j]
 [-0.1767767+0.j]
 [ 0.1767767+0.j]
 [-0.1767767+0.j]
 [ 0.1767767+0.j]
 [-0.1767767+0.j]
 [ 0.1767767+0.j]
 [-0.1767767+0.j]
 [ 0.1767767+0.j]
 [-0.1767767+0.j]]
[[ 0.00000000e+00+0.j]
 [ 5.00000000e-01+0.j]
 [ 0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j]
 [ 5.00000000e-01+0.j]
 [ 0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j]
 [ 5.00000000e-01+0.j]
 [ 0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j]
 [-5.00000000e-01+0.j]
 [ 0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j]
 [ 4.16333634e-17+0.j]
 [ 0.0000

C:\Users\DELL\AppData\Local\Temp\ipykernel_35216\575662396.py:31: ComplexWarning: Casting complex values to real discards the imaginary part
  val = np.array(functools.reduce(lambda x,y: x+y, do_nothing+flip), dtype=int)


TypeError: only length-1 arrays can be converted to Python scalars

In [ ]:
def generate_binary_values(n):
    return [[int(c) for c in bin(i)[2:].zfill(n)] for i in range(2**n)]

# generate arbitrary LINEAR function
n = [2,3,5]  # Input the number of qubits
n = [np.random.randint(low=2,high=5) for _ in range(5)]
print(f"n={n}")
s_n = [[1 if np.random.ranf() < 0.5 else 0 for _ in range(i)] for i in n]
print(f"s_n={s_n}")  # Input the mapping functions

f_map = []
for i,s in enumerate(s_n):
    tmp = []
    for x in range(2**n[i]):
        bin_x = [x >> j & 1 for j in range(n[i])]
        tmp.append(np.dot(bin_x, s) & 1)
    f_map.append(tmp)
print(f"f(0->2^n):{f_map}")

In [ ]:
from qiskit import ClassicalRegister, QuantumRegister

def bernstein_vazirani(n, f: list[bool]):
    print(s)
    nQubits = n
    qr = QuantumRegister(nQubits+1)
    cr = ClassicalRegister(nQubits)
    
    qc = QuantumCircuit(qr, cr)
    # H 
    for q in qr[:-1]:
        qc.h(q)
    qc.x(qr[-1])
    qc.h(qr[-1])
    qc.barrier(qr)

    # Oracle 
    for i in range(nQubits):
        if f[1<<i]:
            qc.cx(qr[i],qr[-1])

    qc.barrier(qr)
    # H
    qc.h(qr[-1])
    qc.x(qr[-1])
    for q in qr[:-1]:
        qc.h(q)

    qc.barrier(qr)
    # Final
    qc.measure(qr[:-1],cr)
    return qc

circuit = bernstein_vazirani(4,[0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0])

style = {'backgroundcolor': 'lavender'}
circuit.draw(output='latex', style = style)

In [ ]:
# use local simulator
backend = AerSimulator()
shots = 1024
job = backend.run(circuits=circuit, shots=shots)
result = job.result()
answer = result.get_counts(0)
plot_histogram(answer)

In [ ]:
num_qubits = 32

alice_basis = np.random.randint(2, size=num_qubits)
alice_state = np.random.randint(2, size=num_qubits)
bob_basis = np.random.randint(2, size=num_qubits)


print(f"Alice's State:\t {np.array2string(alice_state)}")
print(f"Alice's Bases:\t {np.array2string(alice_basis)}")
print(f"Bob's Bases:\t {np.array2string(bob_basis)}")

In [87]:
def bb84_circuit(state, basis, measurement_basis):
   
    #state: array of 0s and 1s denoting the state to be encoded
    #basis: array of 0s and 1s denoting the basis to be used for encoding
                #0 -> Computational Basis
                #1 -> Hadamard Basis
    #meas_basis: array of 0s and 1s denoting the basis to be used for measurement
                #0 -> Computational Basis
                #1 -> Hadamard Basis
    
    num_qubits = len(state)
    
    circuit = QuantumCircuit(num_qubits)

    # Sender prepares qubits
    for i in range(len(basis)):
        if state[i] == 1:
            circuit.x(i)
        if basis[i] == 1:
            circuit.h(i)
   

    # Measuring action performed by Bob
    for i in range(len(measurement_basis)):
        if measurement_basis[i] == 1:
            circuit.h(i)

       
    circuit.measure_all()
    
    return circuit

In [ ]:
circuit = bb84_circuit(alice_state, alice_basis, bob_basis)
# use local simulator
backend = AerSimulator()
shots = 1024
job = backend.run(circuits=circuit, shots=shots)
key = job.result().get_counts(0)
print(key)
encryption_key = ''
for i in range(num_qubits):
    if alice_basis[i] == bob_basis[i]:
         encryption_key += str(key[i])
print(f"Key: {encryption_key}")

In [2]:
def U(n, f: list[bool]):
    assert len(f) == 2**n

    num_qubits = n + 1
    U = np.zeros((2**num_qubits, 2**num_qubits)) 

    for x_y in range(2**num_qubits): 
        x = x_y >> 1 # x
        y_xor_fx = (x_y & 1) ^ (f[x]) # y ^ f(x)
        x_y_xor_fx = (x << 1) + y_xor_fx # x , y^f(x)
        U[x_y, x_y_xor_fx] = 1

    return U

In [ ]:
f=[0,1,0,1]
n=2
u = U(n,f)
print(u)

In [ ]:
state_list = [KET_0, KET_0, KET_0]
test = functools.reduce(lambda x,y: np.kron(x,y), state_list)
print(u @ test) 

In [ ]:
def to_bin(d) : return [int(j) for j in f'{d:0{2**n}b}']
def trans(mat): return [[c] for c in mat]
y = 0
yxorf = 0 ^ f[y]
print (np.array(trans(to_bin(2)+to_bin(y))))
for i in range(len(f)):
    out = np.dot (u, np.array(to_bin(i)+to_bin(y),dtype=int) )
    expect = np.array(to_bin(i) + to_bin(yxorf))
    print(out)
    print(expect)


In [ ]:
print(np.dot(H,H))

In [ ]:
from deustch_josza import deutsch_algorithm

n = [2,3,3]  # Input the number of qubits
f_map = [[0,0,1,1],
            [1,1,1,1,1,1,1,1],
            [1,0,0,1,1,0,1,0]]  # Input the mapping functions
for index, value in enumerate(n):
    deutsch_algorithm(n[index], f_map[index])  # Algorithm executed here